# IMPORT LIBARY

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import tkinter as tk 

from tkinter import ttk,messagebox  

In [ ]:
x = pd.read_csv('datasets/smartphone_battery_features.csv')
y = pd.read_csv('datasets/smartphone_battery_targets.csv')
df = pd.concat([x, y], axis=1)

df.head()

In [ ]:
# Cek Tipe Data Pada Setiap Kolom
df.info()

In [ ]:
dimension = df.shape 
print(f"Dimensi Dari Dataset : {dimension}")

MENETUKAN NAMA FITUR

In [ ]:
df.columns

In [ ]:
df.describe()

MENANGANI DATA YANG BERMASALAH

In [ ]:
count_duplicated = df.duplicated().sum()
print(f'Jumlah Data Duplikat : {count_duplicated}')

count_isna = df.isna().sum().sum()
print(f"Jumlah Data Kosong Pada Dataset : {count_isna}")

PENANGANAN APABILA TERDAPAT MISSING VALUE

In [ ]:
if count_isna > 0:
    for x in df.columns:
        if df[x].dtype == 'Object':
            df[x] = df[x].fillna().mode()[0]
        else:
            df[x] = df[x].fillna().mean()

else :
    print("Tidak Ada Data Yang Perlu Ditangani")

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.drop(columns=['Device_ID'])

In [ ]:
df.head(10)

MENENTUKAN LABEL X DAN Y

In [ ]:
# background_app_usage_level 
# signal_strengt_avg 
# recomended_action

In [ ]:
df['signal_strength_avg'] = df['signal_strength_avg'].replace({
    'Low' : 0,
    'Medium' : 1,
    'High' : 2
})
df['charging_habit_score'] = df['charging_habit_score'].replace({
    'Poor' : 0,
    'Moderate' : 1,
    'Good' : 2
})
df['recommended_action'] = df['recommended_action'].replace({
    'Change Phone' : 0,
    'Replace Battery' : 1,
    'Keep Using' : 2
})

In [ ]:
df.head(10)

MENENTUKAN FITUR DAN LABEL (X DAN Y)

In [ ]:
x = [['device_age_month', 'avg_charging_per_week', 'avg_battery_temp_celcius', 'charging_habbit_score']]
y1 = ['current_battery_health_percent']
y2 = ['recommended_action']

In [ ]:
x = df[[]]

# EXPLORATORY DATA ANALYS | MODUL A

STATISTIK DESKRIPTIF

In [ ]:
for i in df.columns:
    sns.histplot(df[i], kde=True)
    plt.show()

In [ ]:
for i in df.columns:
    sns.boxplot(df[i])
    plt.show()

In [ ]:
kolom = ['gaming_hours_per_week']
def outliers(df, kolom):
    for col in kolom:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)

        IQR = Q3 - Q1

        atas = Q3 + 1.5 * IQR
        bawah = Q1 + 1.5 * IQR

        df[col] = df[col].clip(upper=18, lower=0) #Kami menggunakan metode clip
    return df

In [ ]:
df = outliers(df, kolom=kolom)


In [ ]:
for col in df.columns:
    sns.boxplot(df[col])
    plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True)
plt.show()

# MODUL B | DATA CLASSIFICATION LINEAR REGRESION


train data split

In [ ]:
test_data = df.sample(frac=0.2, random_state=42)
train_data = df.drop(test_data.index)

X_train = train_data.drop(columns=['current_battery_health_percent', 'recommended_action'])
X_test = test_data.drop(columns=['current_battery_health_percent', 'recommended_action'])

y_test_bh = test_data['current_battery_health_percent']
y_train_bh = train_data['current_battery_health_percent']

y_test_ra = test_data['recommended_action']
y_train_ra = train_data['recommended_action']

In [ ]:
print(f"Dimension dari data y test bh {y_test_bh.shape}")
print(f"Dimension dari data y train bh {y_train_bh.shape}")
print(f"Dimension dari data y test ra {y_test_ra.shape}")
print(f"Dimension dari data y test ra {y_train_ra.shape}")

In [ ]:
x_tr_std = X_train.std()
x_tr_mean = X_train.mean()
X_train = (X_train - x_tr_mean) / x_tr_std
X_test = (X_test - x_tr_mean) / x_tr_std

In [ ]:
class LinearRegresion:
    def __init__(self):
        self.beta = None
    
    def fit(self, x_tr, y_tr):
        x_tr_b = np.column_stack([np.ones(x_tr.shape[0]), x_tr])
        b = np.linalg.inv(x_tr_b.T @ x_tr_b) @ x_tr_b.T @ y_tr
        print(f'beta adalah {b}')
        self.beta = b
        
    def predict(self, x):
        x_b = np.column_stack([np.ones(x.shape[0]), x]) 
        return x_b @ self.beta

TRAIN DATA

In [ ]:
model = LinearRegresion()
model.fit(X_train, y_train_bh)

In [ ]:
pred = model.predict(X_test)
pred_1 = model.predict(X_train)

In [ ]:
def define_linear(age, charging, battery_temp, charging_habbit):
    X = np.array([[age, charging, battery_temp, charging_habbit]])
    return model.predict(X)[0]

In [ ]:
def confussion(y,y_pred):
    labels = ['Change Phone','Keep Using','Replace Battery']
    n = len(labels)
    cm = np.zeros([n,n], dtype=int)

    akurasi = np.mean(y.values==y_pred)
    print(f'Akurasi : {akurasi:.3f}')

    for a, p in zip(y,y_pred):
        cm[a][p] += 1

    for i in range(n):
        tp = cm[i,i].sum()
        fp = cm[:,i].sum() - tp
        fn = cm[i,:].sum() - tp

        presicion = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        print(f'{labels[i]} : presicion = [{presicion:.3f}], recall = [{recall:.3f}]')

    plt.figure(figsize=(15,10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='coolwarm', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Prediksi')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
def evaluasi(y, y_pred):
    r2 = 1 - (np.sum((y - y_pred)**2)) / (np.sum((y-y.mean())**2))
    mae = np.mean(np.abs(y-y_pred))
    mse = np.mean((y-y_pred)**2)
    rmse = np.sqrt(mse)
    print(f'R2 : {r2}')
    print(f'MAE : {mae}')
    print(f'MSE : {mse}')
    print(f'RMSE : {rmse}')
    print(f'Range Battery Health (BH): {y.min()}-{y.max()} ') 

In [ ]:
evaluasi(y_test_bh, pred)

In [ ]:
evaluasi(y_train_bh, pred_1)

In [ ]:
def mapping(y_pred):
    label = []
    for i in y_pred:
        if i >= 75: # Jika hasil prediksi diatas 75%
            label.append(2) # Labelnya keep using
        elif i >= 50 and i <= 74: # Jika berada di rentang 50-74%
            label.append(1) #Labelnya Replace Battery
        else: # Jika Hasil prediksi dibawah 50%
            label.append(0) #Labelnya Change phone

    return label #Mengembalikan 

In [ ]:
y_act_tr_pred = mapping(pred)
y_act_te_pred = mapping(pred_1)

In [ ]:
confussion(y_test_ra, y_act_tr_pred)

In [ ]:
def plot_regresi(y, y_pred):
    plt.scatter(x=y, y=y_pred)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], '--r')
    plt.xlabel('Actual Battery Health')
    plt.ylabel('Predicted Battery Health')
    plt.show()

    residual = y - y_pred

    plt.scatter(x=y_pred, y=residual)
    plt.axhline(0, linestyle='--', color='red')
    plt.xlabel('Actual Battery Health')
    plt.ylabel('Predicted Battery Health')
    plt.show()

    sns.histplot(data=residual)
    plt.show()

In [ ]:
plot_regresi(y_test_bh, pred)

In [ ]:
plot_regresi(y_train_bh, pred_1)

# GUI | MODUL D

In [ ]:
import tkinter as tk 
from tkinter import ttk,messagebox 

class GUI:
    def __init__(self, root, tabCtrl):
        self.root = root 
        self.root.geometry("550x750")
        self.root.title("GUI PREDIKSI BATTERY HEALTH")
        self.tabCtrl = tabCtrl 
        self.bg = '#e7eaeb'
        self.ui_index()

    def ui_index(self):
        tab1 = tk.Frame(self.tabCtrl, bg=self.bg)
        self.tabCtrl.add(tab1, text="Prediksi Battery Health")
        header = tk.Frame(tab1, width=50, borderwidth=1, relief='solid')
        header.pack(fill='x', padx=10, pady=20)

        tk.Label(header, text="CEK BATTERY HEALTH", font=("Segoe UI", 10, "bold")).pack(pady=20)

        form_container = tk.Frame(tab1, bg=self.bg)
        form_container.pack(fill='y', padx=10, pady=20)

        dvc_age = tk.Label(form_container, bg=self.bg, text="Masukan Usia Device", font=("Segoe UI", 10, "bold"))
        dvc_age.pack(pady=20)       
        self.entry_dvc_age = tk.Entry(form_container, width=40)
        self.entry_dvc_age.pack(pady=5)

        avg_charging = tk.Label(form_container, bg=self.bg, text="Rata-rata Charging Tiap Mingu", font=("Segoe UI", 10, "bold"))
        avg_charging.pack(pady=20)       
        self.entry_avg_charging = tk.Entry(form_container, width=40)
        self.entry_avg_charging.pack(pady=5)


        avg_battery_temp = tk.Label(form_container, bg=self.bg, text="Temperature Batterai", font=("Segoe UI", 10, "bold"))
        avg_battery_temp.pack(pady=20)       
        self.entry_avg_battery_temp = tk.Entry(form_container, width=40)
        self.entry_avg_battery_temp.pack(pady=5)


        charging_habbit = tk.Label(form_container, bg=self.bg, text="Charging Habit Score", font=("Segoe UI", 10, "bold"))
        charging_habbit.pack(pady=20)       
        self.entry_charging_habbit = tk.Entry(form_container, width=40)
        self.entry_charging_habbit.pack(pady=5)

        tk.Button(tab1, width=37, text="SUBMIT", bg="#6fd889", fg='white', font=("Segoe UI", 10, "bold"), relief='groove', command=self.analys).pack()


    def analys(self): 
        age = self.entry_dvc_age.get()
        charging = self.entry_avg_charging.get()
        battery_temp = self.entry_avg_battery_temp.get()
        charging_habbit = self.entry_charging_habbit.get()

        if age == "" or charging == "" or battery_temp == "" or charging_habbit == "":
            messagebox.showerror("error", "Mohon isi semua inputan") 
        else :
            try :
                kesehatan, aksi = define_linear(
                    age,
                    charging,
                    battery_temp,
                    charging_habbit
                )

                messagebox.showinfo("Informasi Device Anda", {kesehatan}, {aksi})
            except ValueError:
                messagebox.showerror("Terjadi Kesalahan Saat Memproses Data")


if __name__ == "__main__":
    root = tk.Tk()
    tabCtrl = ttk.Notebook(root)
    tabCtrl.pack(expand=True, fill='both')
    GUI(root, tabCtrl)
    root.mainloop()
    

